# Naver blog scrap

In [2]:
first_list = [ [] ] * 10
len(first_list), first_list

(10, [[], [], [], [], [], [], [], [], [], []])

In [3]:
date_list = first_list[0]
id(date_list)

2191901671232

In [5]:
rare_number_list  = first_list[1]
id(rare_number_list )

2191901671232

In [6]:
date_list.append(1)
date_list, rare_number_list

([1], [1])

## 라이브러리

In [75]:
import pandas as pd
import time, os, random
from datetime import datetime
from numpy import nan

In [76]:
from bs4 import BeautifulSoup as bs
from selenium import webdriver
import chromedriver_autoinstaller as ca

## 함수

In [123]:
def crawling(comment_areas, list):
    # 리스트 지정
    date_list = list[0]
    rare_number_list = list[1]
    check_reply_list = list[2]
    nickname_list = list[3]
    writer_url_list = list[4]
    like_num_list = list[5]
    link_num_list = list[6]
    reply_num_list = list[7]
    img_list = list[8]
    comment_list = list[9]

    for one in comment_areas:
        # 작성일 수집
        date = one.find('div', 'u_cbox_comment_box').find('span','u_cbox_date').attrs['data-value'].split('+')[0].replace('T',' ')
        date_list.append(date)

        # 고유번호
        temp = one.attrs['class'][1].split('_')[-1]
        rare_number_list.append(temp)

        # 답글 구분
        try:
            one.find('span', 'u_cbox_secret_contents').text
            temp = '비밀댓글'
        except:
            try:
                temp = one.find('div', 'u_cbox_reply_area').find('li')
                temp = nan
            except:
                temp = '답글'
        check_reply_list.append(temp)

        # 닉네임
        try:
            temp = one.find('span', 'u_cbox_nick').text
        except:
            temp = '비밀댓글'
        nickname_list.append(temp)

        # 작성자 url
        try:
            temp = one.find('a', 'u_cbox_name').attrs['href']
        except:
            temp = '비밀댓글'
        writer_url_list.append(temp)

        # 공감수
        try:
            temp = int( one.find('em', 'u_cbox_cnt_recomm').text )
        except:
            temp = 0
        like_num_list.append(temp)

        # 링크 수
        try:
            temp = one.find('span', 'u_cbox_contents').find_all('a', 'u_cbox_contents_link')
            temp = len(temp)
        except:
            temp = 0
        link_num_list.append(temp)

        # 답글 수
        try:
            temp = one.find('div', 'u_cbox_reply_area').find_all('li')
            temp = len(temp)
        except:
            temp = 0
        reply_num_list.append(temp)

        # 첨부이미지url
        try:
            temp = one.find('div', 'u_cbox_image_section').find('img').attrs['src']
        except:
            temp = nan
        img_list.append(temp)

        # 댓글 내용 수집
        try:
            temp = one.find('span', 'u_cbox_contents').text.replace('@', ' @', 1).replace('http', ' http', 1)
        except:
            temp = '비밀댓글'
        comment_list.append(temp)

    # 수집한 데이터 확인
    print('작성일 리스트 길이 : ', len(date_list))
    print('고유번호 리스트 길이 : ', len(rare_number_list))
    print('답글구분 리스트 길이 : ', len(check_reply_list))
    print('닉네임 리스트 길이 : ', len(nickname_list))
    print('작성자URL 리스트 길이 : ', len(writer_url_list))
    print('공감수 리스트 길이 : ', len(like_num_list))
    print('링크수 리스트 길이 : ', len(link_num_list))
    print('답글수 리스트 길이 : ', len(reply_num_list))
    print('첨부이미지URL 리스트 길이 : ', len(img_list))
    print('내용 리스트 길이 : ', len(comment_list))

    # 리스트에 담기
    crawled_list = [date_list, rare_number_list, check_reply_list, nickname_list, 
            writer_url_list, like_num_list, link_num_list, reply_num_list,
            img_list, comment_list]

    return crawled_list

## 현재 파일 위치 확인

In [77]:
# 현재 파일 위치 확인
code_path = os.getcwd().replace('\\', '/')
code_path

'c:/data/Wantreez/Crawling/incheon'

## 수집한 파일 저장 폴더 생성

In [78]:
# 수집한 파일을 저장할 폴더 생성
crawled_folder_path = code_path + '/crawled_data/naver_blog'
os.makedirs(crawled_folder_path, exist_ok=True)

## 크롬 드라이버 버전 확인 및 설치

In [79]:
# 크롬 드라이버 버전 확인
chrome_ver = ca.get_chrome_version().split('.')[0]
chrome_ver

'100'

In [125]:
# 크롬 드라이버 설치 확인
try:
    driver = webdriver.Chrome(code_path + f'/{chrome_ver}/' + 'chromedriver.exe')
except:
    ca.install(True)
    driver = webdriver.Chrome(code_path + f'/{chrome_ver}/' + 'chromedriver.exe')

C:\Users\kbjung\AppData\Local\Temp/ipykernel_7724/3261653714.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(code_path + f'/{chrome_ver}/' + 'chromedriver.exe')


## 페이지 접속

In [ ]:
# 댓글 1페이지 : https://blog.naver.com/PostView.naver?blogId=icouncil103&logNo=222668964493&categoryNo=9&parentCategoryNo=9&from=thumbnailList

# 댓글 5페이지 : https://blog.naver.com/PostView.naver?blogId=icouncil103&logNo=222579280294&categoryNo=9&parentCategoryNo=9&from=thumbnailList

# 댓글 15페이지 : https://blog.naver.com/PostView.naver?blogId=icouncil103&logNo=222657505998&categoryNo=9&parentCategoryNo=9&from=thumbnailList

In [126]:
# 페이지 접속
url = 'https://blog.naver.com/PostView.naver?blogId=icouncil103&logNo=222657505998&categoryNo=9&parentCategoryNo=9&from=thumbnailList'
driver.get(url)
driver.implicitly_wait(3)
driver.maximize_window()

In [127]:
# page 정보
page_info = url.split('=')[2].split('&')[0]
page_info

'222657505998'

## 페이지 스크래핑

### 댓글 버튼 클릭

In [128]:
# 댓글 버튼 클릭
comment_button = driver.find_element_by_xpath(f'//*[@id="Comi{page_info}"]')
comment_button.click()
time.sleep(1)

C:\Users\kbjung\AppData\Local\Temp/ipykernel_7724/3197372586.py:2: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  comment_button = driver.find_element_by_xpath(f'//*[@id="Comi{page_info}"]')


## 페이지 이동 규칙 찾기

In [129]:
# 이전페이지 : //*[@id="naverComment_201_222657505998"]/div/div[4]/div/a[2]
# 1 : //*[@id="naverComment_201_222657505998"]/div/div[4]/div/a[2]
# 2 : //*[@id="naverComment_201_222657505998"]/div/div[4]/div/a[1]
# 3 : //*[@id="naverComment_201_222657505998"]/div/div[4]/div/a[3]
# 4 : //*[@id="naverComment_201_222657505998"]/div/div[4]/div/a[4]
# 5 : //*[@id="naverComment_201_222657505998"]/div/div[4]/div/a[5]
# 6 : //*[@id="naverComment_201_222657505998"]/div/div[4]/div/a[6]
# 7 : //*[@id="naverComment_201_222657505998"]/div/div[4]/div/a[7]
# 8 : //*[@id="naverComment_201_222657505998"]/div/div[4]/div/a[8]
# 9 : //*[@id="naverComment_201_222657505998"]/div/div[4]/div/a[9]
# 10 : //*[@id="naverComment_201_222657505998"]/div/div[4]/div/a[10]
# 11(다음페이지) : //*[@id="naverComment_201_222657505998"]/div/div[4]/div/a[11]
# 12 : //*[@id="naverComment_201_222657505998"]/div/div[4]/div/a[3]
# 13 : //*[@id="naverComment_201_222657505998"]/div/div[4]/div/a[4]
# 14 : //*[@id="naverComment_201_222657505998"]/div/div[4]/div/a[5]
# 15 : //*[@id="naverComment_201_222657505998"]/div/div[4]/div/a[6]

### 페이지 버튼 개수

In [130]:
soup = bs(driver.page_source, 'lxml')
num_page_buttons = int(soup.find('span', '_lastPageNo').text)
num_page_buttons

15

In [131]:
list = [[], [], [], [], [], [], [], [], [], []]
len(list), list

(10, [[], [], [], [], [], [], [], [], [], []])

In [132]:
for i in range(num_page_buttons - 1):
    button = driver.find_element_by_xpath(f'//*[@id="naverComment_201_{page_info}_ct"]/div[1]/div/div[2]/a[1]')
    try:
        # 페이지 이동
        button.click()
        driver.implicitly_wait(1)
        time.sleep( random.uniform(0.5, 1) )
        # 페이지 수프에 담기
        comment_page_soup = bs(driver.page_source, 'lxml')
        comment_areas = comment_page_soup.find_all('li', 'u_cbox_comment')
        print( len(comment_areas) )
        # 정보 크롤링
        list = crawling(comment_areas, list)
    except:
        pass

C:\Users\kbjung\AppData\Local\Temp/ipykernel_7724/1262821627.py:2: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  button = driver.find_element_by_xpath(f'//*[@id="naverComment_201_{page_info}_ct"]/div[1]/div/div[2]/a[1]')


50
작성일 리스트 길이 :  50
고유번호 리스트 길이 :  50
답글구분 리스트 길이 :  50
닉네임 리스트 길이 :  50
작성자URL 리스트 길이 :  50
공감수 리스트 길이 :  50
링크수 리스트 길이 :  50
답글수 리스트 길이 :  50
첨부이미지URL 리스트 길이 :  50
내용 리스트 길이 :  50
50
작성일 리스트 길이 :  100
고유번호 리스트 길이 :  100
답글구분 리스트 길이 :  100
닉네임 리스트 길이 :  100
작성자URL 리스트 길이 :  100
공감수 리스트 길이 :  100
링크수 리스트 길이 :  100
답글수 리스트 길이 :  100
첨부이미지URL 리스트 길이 :  100
내용 리스트 길이 :  100
50
작성일 리스트 길이 :  150
고유번호 리스트 길이 :  150
답글구분 리스트 길이 :  150
닉네임 리스트 길이 :  150
작성자URL 리스트 길이 :  150
공감수 리스트 길이 :  150
링크수 리스트 길이 :  150
답글수 리스트 길이 :  150
첨부이미지URL 리스트 길이 :  150
내용 리스트 길이 :  150
50
작성일 리스트 길이 :  200
고유번호 리스트 길이 :  200
답글구분 리스트 길이 :  200
닉네임 리스트 길이 :  200
작성자URL 리스트 길이 :  200
공감수 리스트 길이 :  200
링크수 리스트 길이 :  200
답글수 리스트 길이 :  200
첨부이미지URL 리스트 길이 :  200
내용 리스트 길이 :  200
50
작성일 리스트 길이 :  250
고유번호 리스트 길이 :  250
답글구분 리스트 길이 :  250
닉네임 리스트 길이 :  250
작성자URL 리스트 길이 :  250
공감수 리스트 길이 :  250
링크수 리스트 길이 :  250
답글수 리스트 길이 :  250
첨부이미지URL 리스트 길이 :  250
내용 리스트 길이 :  250
50
작성일 리스트 길이 :  300
고유번호 리스트 길이 :  300
답글구분 리스트 길

In [133]:
print('작성일 리스트 길이 : ', len(list[0]))
print('고유번호 리스트 길이 : ', len(list[1]))
print('답글구분 리스트 길이 : ', len(list[2]))
print('닉네임 리스트 길이 : ', len(list[3]))
print('작성자URL 리스트 길이 : ', len(list[4]))
print('공감수 리스트 길이 : ', len(list[5]))
print('링크수 리스트 길이 : ', len(list[6]))
print('답글수 리스트 길이 : ', len(list[7]))
print('첨부이미지URL 리스트 길이 : ', len(list[8]))
print('내용 리스트 길이 : ', len(list[9]))

작성일 리스트 길이 :  701
고유번호 리스트 길이 :  701
답글구분 리스트 길이 :  701
닉네임 리스트 길이 :  701
작성자URL 리스트 길이 :  701
공감수 리스트 길이 :  701
링크수 리스트 길이 :  701
답글수 리스트 길이 :  701
첨부이미지URL 리스트 길이 :  701
내용 리스트 길이 :  701


## df 만들기

In [134]:
dict = {'작성일':date_list, '고유번호':rare_number_list, 
        '답글구분':check_reply_list, '닉네임':nickname_list, 
        '작성자URL':writer_url_list, '공감수':like_num_list, 
        '링크수':link_num_list, '답글수':reply_num_list, 
        '첨부이미지URL':img_list, '내용':comment_list}
df = pd.DataFrame(dict)

In [135]:
df.head()

,작성일,고유번호,답글구분,닉네임,작성자URL,공감수,링크수,답글수,첨부이미지URL,내용
0,2022-02-27 17:29:16,undefined,NaN,란나,https://blog.naver.com/kiboda,0,0,3,https://g-cbox.pstatic.net/MjAyMjAyMjdfMTY1/MD...,
1,2022-02-27 17:29:16,750830888173961351,답글,란나,https://blog.naver.com/kiboda,0,0,0,https://g-cbox.pstatic.net/MjAyMjAyMjdfMTY1/MD...,
2,2022-02-27 17:29:28,750830900253556981,답글,란나,https://blog.naver.com/kiboda,0,0,0,https://g-cbox.pstatic.net/MjAyMjAyMjdfMTQ3/MD...,
3,2022-02-27 17:29:38,750830912618365258,답글,란나,https://blog.naver.com/kiboda,0,0,0,https://g-cbox.pstatic.net/MjAyMjAyMjdfNDUg/MD...,
4,2022-02-27 17:31:13,750831012895785085,NaN,lhj123400,https://blog.naver.com/lhj123400,0,1,0,NaN,정답 : 3번 https://blog.naver.com/lhj123400/22265...


## 파일로 출력

In [629]:
crawled_folder_path

'c:/data/Wantreez/Crawling/incheon/crawled_data/naver_blog'

In [630]:
today_date = datetime.today().strftime('%Y%m%d_%H%M%S')
today_date

'20220415_133755'

In [631]:
today_date = datetime.today().strftime('%Y%m%d_%H%M%S')
file_name = f'naver_blog_{today_date}.xlsx'
df.to_excel(crawled_folder_path + '/' + file_name, index=False, encoding='utf-8')
print(f'{file_name} 출력완료')

naver_blog_20220415_133755.xlsx 출력완료


# 코드 마지막